In [1]:
class classes:
    classes_MCD12Q1v6LCProp2 = {0: 'No data',  1: 'Barren',
     2: 'Permanent Snow and Ice',
     3: 'Water Bodies',
     4: 'Urban and Built-up Lands',
     5: 'Dense Forests',
     6: 'Open Forests',
     7: 'Forest/Cropland Mosaics',
     8: 'Natural Herbaceous',
     9: 'Natural Herbaceous-Croplands Mosaics',
     10: 'Herbaceous Croplands',
     11: 'Shrublands'}

    classes_MCD12Q1v6LCProp2_major = {
     0: 'No data',  
     1: 'Barren',
     2: 'Water Bodies',
     3: 'Urban and Built-up Lands',
     4: 'Dense Forests',
     5: 'Open Forests',
     6: 'Natural Herbaceous',
     7: 'Herbaceous Croplands',
     8: 'Shrublands'}
        

In [13]:
import numpy as np
import os
from collections import Counter
import pandas as pd

In [29]:
ps_train = 24
ps_eval = 384
project = "AMZ"

In [30]:
indir = r"F:\acoca\research\gee\dataset\{}\gz\{}\multiple\tileids".format(project, ps_train)

testtiles = np.loadtxt(os.path.join(indir,"test_fold0_sample.tileids"), dtype='str')
traintiles = np.loadtxt(os.path.join(indir,"train_fold0_sample.tileids"), dtype='str')

In [31]:
indir = r"F:\acoca\research\gee\dataset\{}\gz\{}\multiple\tileids".format(project,ps_eval)

evaltiles = np.loadtxt(os.path.join(indir,"eval.tileids"), dtype='str')

In [32]:
def count_split(tiles, indir, reference):
    label_all = []
    tilesid=tiles
    for i in range(1, 2):
        indir_y = os.path.join(indir,"data"+str(i).zfill(2),reference)
        
        if len(tilesid.flatten()) > 1:
            label_y = []

            for f in range(0,len(tilesid)):
                    filename = os.path.join(indir_y,tilesid[f]+".npy")
                    labels__ = np.load(filename)
                    label_y = np.append(label_y,labels__)

            label_all.append(label_y)
        else:
            filename = os.path.join(indir_y,"{}.npy".format(tilesid))
            label_all = np.load(filename)

    return(label_all)

In [33]:
classdir = r"F:\acoca\research\gee\dataset\{}\gz\{}\multiple\classes".format(project,ps_train)
classdir_eval = r"F:\acoca\research\gee\dataset\{}\gz\{}\multiple\classes".format(project,ps_eval)

scheme_clas = 'MCD12Q1v6stable01to03_LCProp2_major'

train_all = count_split(traintiles, classdir,scheme_clas)
test_all = count_split(testtiles, classdir,scheme_clas)
eval_all = count_split(evaltiles, classdir_eval,scheme_clas)

In [34]:
d_train = pd.Series(train_all[0])
d_test = pd.Series(test_all[0])
if len(eval_all) == 1:
    d_eval = pd.Series(eval_all[0])
else:
    d_eval = pd.Series(eval_all)

In [35]:
vc_train = d_train.value_counts(normalize=False)
vc_test = d_test.value_counts(normalize=False)
vc_eval = d_eval.value_counts(normalize=False)

In [37]:
all_data = pd.concat((vc_train,vc_test,vc_eval), axis=1)
all_data = all_data.rename(columns={0: 'train', 1: 'test', 2: 'evaluation'})
all_data['class_id'] = all_data.index
all_data['class'] = all_data['class_id'].map(classes.classes_MCD12Q1v6LCProp2_major)
all_data

,train,test,evaluation,class_id,class
0.0,605997,166767,304501,0.0,No data
1.0,528,129,5532,1.0,Barren
2.0,256679,28553,108068,2.0,Water Bodies
3.0,24410,3311,10092,3.0,Urban and Built-up Lands
4.0,4601962,1725597,2420679,4.0,Dense Forests
5.0,5684705,957342,2656892,5.0,Open Forests
6.0,2265682,580530,1039406,6.0,Natural Herbaceous
7.0,256067,75961,85597,7.0,Herbaceous Croplands
8.0,17378,754,4753,8.0,Shrublands


## weights

In [107]:
#remove non-reliable
newarray = [x -1 for x in train_all[0]]
newarray = [x for x in newarray if x != -1]

In [108]:
category_train = np.array(newarray)
n_classes=8
import math
from collections import Counter
counts = Counter(category_train)
weightsArray =[]
for i in range(n_classes):
    weightsArray.append(math.log(category_train.shape[0]/max(counts[i],1))+1)

In [109]:
weightsArray

[12.566200793829088,
 12.566200793829088,
 8.159481546564836,
 9.793612071589308,
 1.1665112184715174,
 3.4213594040856017,
 12.566200793829088,
 3.7542506162891085]